In [2]:
# Load all the required libraries

import pyedflib
import numpy as np
from scipy import signal
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras 
import tensorflow as tf
import os, datetime
import seaborn as sns
import gdown

In [2]:
from models.model import Model_EEG

In [11]:
# Load train and test datasets.

train_url = 'https://drive.google.com/uc?id=1z5IhGmTQmqPJ7SO7aAni1JJyJHnmWinp'
test_url = 'https://drive.google.com/uc?id=16LYmLmOb--tXGCAn3YMIetSqGKgmB-b8'
train_ = 'df_train.json'
test_ = 'df_test.json'

gdown.download(train_url, train_, quiet=False)
gdown.download(test_url, test_, quiet=False)

df_train = pd.read_json(train_)
df_test = pd.read_json(test_)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1z5IhGmTQmqPJ7SO7aAni1JJyJHnmWinp
From (redirected): https://drive.google.com/uc?id=1z5IhGmTQmqPJ7SO7aAni1JJyJHnmWinp&confirm=t&uuid=41356bde-7a52-4523-b2a2-dd09f7d9ac69
To: c:\Users\dheer\WHK\EEG-Non_EEG-XAI-analysis-main\EEG-Non_EEG-XAI-analysis-main\EEG\df_train.json
100%|██████████| 246M/246M [02:07<00:00, 1.92MB/s] 
Downloading...
From: https://drive.google.com/uc?id=16LYmLmOb--tXGCAn3YMIetSqGKgmB-b8
To: c:\Users\dheer\WHK\EEG-Non_EEG-XAI-analysis-main\EEG-Non_EEG-XAI-analysis-main\EEG\df_test.json
100%|██████████| 6.76M/6.76M [00:03<00:00, 1.75MB/s]


In [12]:
df_train

,Data,Label
0,"[[-9.1819291819, 0.19536019540000002, 18.16849...",0
1,"[[-39.2673992674, -27.5457875458, 29.890109890...",0
2,"[[-10.3540903541, 12.6984126984, 33.7973137973...",0
3,"[[-34.188034188, 19.3406593407, 2.539682539700...",0
4,"[[45.9096459096, 50.5982905983, -1.7582417582,...",0
...,...,...
2905,"[[-10.3540903541, 58.4126984127, 28.7179487179...",1
2906,"[[-23.2478632479, 61.1477411477, -15.824175824...",1
2907,"[[-1.7582417582, 2.5396825397000002, 43.956043...",1
2908,"[[-13.0891330891, 48.253968254, 31.8437118437,...",1


In [5]:
# Prepare train and validation data

data = np.array([np.array(xi, dtype='float32') for xi in df_train['Data']])
labels = np.asarray(df_train['Label'])

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.3, random_state=1, shuffle=True)

batch_size = 64 #min(x_train.shape[0]/10, 16)
nb_classes = len(np.unique(y_val))
y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
y_val = (y_val - y_val.min())/(y_val.max()-y_val.min())*(nb_classes-1)   

Y_train = keras.utils.to_categorical(y_train, nb_classes)
Y_val = keras.utils.to_categorical(y_val, nb_classes)
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean)/(x_train_std)
x_val = (x_val - x_train_mean)/(x_train_std)
x_train = x_train.reshape(x_train.shape + (1,))
x_val = x_val.reshape(x_val.shape + (1,))
x_val_std = x_val.std()
x_val_mean = x_val.mean()

In [ ]:
cpdir = os.path.join("checkpoints", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
checkpoint_filepath = cpdir
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, 
                                                            monitor='val_accuracy', save_weights_only=True, 
                                                            mode='max', save_best_only=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor=0.5, patience=10, min_lr=0.0001) 

early_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=0, mode='max',
                                           baseline=None, restore_best_weights=False)

moddir = os.path.join("EEG/saved-model/", 
                      datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [69]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)        
model = Model_EEG(x_train, nb_classes)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(x_train, Y_train, batch_size=batch_size, epochs=50, 
              validation_data=(x_val, Y_val), callbacks = [early_stop, reduce_lr,
                                                                      model_checkpoint_callback])
model.load_weights(cpdir)
model.save(moddir+'-'+str(i), save_format='h5')

Epoch 1/50
32/32 [==============================] - 1078s 34s/step - loss: 0.5185 - accuracy: 0.7531 - val_loss: 0.8102 - val_accuracy: 0.4960 - lr: 1.0000e-04
Epoch 2/50
32/32 [==============================] - 1084s 34s/step - loss: 0.3767 - accuracy: 0.8375 - val_loss: 1.1778 - val_accuracy: 0.4960 - lr: 1.0000e-04
Epoch 3/50
32/32 [==============================] - 1089s 34s/step - loss: 0.3443 - accuracy: 0.8552 - val_loss: 1.5086 - val_accuracy: 0.4960 - lr: 1.0000e-04
Epoch 4/50
32/32 [==============================] - 1085s 34s/step - loss: 0.3215 - accuracy: 0.8768 - val_loss: 1.9207 - val_accuracy: 0.4960 - lr: 1.0000e-04
Epoch 5/50
32/32 [==============================] - 1088s 34s/step - loss: 0.3029 - accuracy: 0.8846 - val_loss: 2.2469 - val_accuracy: 0.4960 - lr: 1.0000e-04
Epoch 6/50
32/32 [==============================] - 1075s 34s/step - loss: 0.3015 - accuracy: 0.8807 - val_loss: 2.2909 - val_accuracy: 0.4960 - lr: 1.0000e-04
Epoch 7/50
32/32 [======================

In [8]:
model